<a href="https://colab.research.google.com/github/KenObata/TISMIR_notebooks/blob/main/Glove_XGB_KFold_kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## This notebook uses Glove word matrix-> SMOTE. 

Dataset: kaggle 250,000 songs
Situation: English only (=multi-class).

Split: StratifiedKfold.



### set up

In [1]:
!pip3 install scikit-multilearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 7.7 MB/s 


In [2]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
from collections import Counter

from skmultilearn.model_selection import IterativeStratification   
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import csr_matrix

### Data Preparation(Kfold split)

Create dataframe for Gensim

In [3]:
import pandas as pd

DIR = '/content/drive/MyDrive/music4all/'
df_genre_by_lang = pd.read_csv(DIR + 'kaggle/df_kaggle.csv')
df_genre_by_lang = df_genre_by_lang.drop(columns = ['Unnamed: 0'])
df_genre_by_lang

,Genre,Lyrics
0,Hip-Hop,Most folks spend their days daydreaming of fin...
1,Indie,Take your cold hands and put them on my face\n...
2,Metal,Are you ready it's time for war\nWe'll break d...
3,Pop,You ask me why I change the color of my hair\n...
4,Hip-Hop,Do you believe in magic in a young girl's hear...
...,...,...
258117,R&B,I'm the best friend he's got I'd give him the ...
258118,Pop,"Bad Boys Blue ""I Totally Miss You"" I did you w..."
258119,Pop,Forgive me for the things That I never said to...
258120,Indie,The day they found a cure for AIDS The day the...


In [4]:
def load_data(df_col, y):
    texts, labels = [], []
    
    for line in df_col:
        # texts are already tokenized, just split on space
        # in a real use-case we would put more effort in preprocessing
        texts.append(line.split(' '))
    return pd.DataFrame({'texts': texts, 'labels': y})

In [5]:
data = load_data(df_genre_by_lang["Lyrics"], df_genre_by_lang["Genre"])

In [6]:
data

,texts,labels
0,"[Most, folks, spend, their, days, daydreaming,...",Hip-Hop
1,"[Take, your, cold, hands, and, put, them, on, ...",Indie
2,"[Are, you, ready, it's, time, for, war\nWe'll,...",Metal
3,"[You, ask, me, why, I, change, the, color, of,...",Pop
4,"[Do, you, believe, in, magic, in, a, young, gi...",Hip-Hop
...,...,...
258117,"[I'm, the, best, friend, he's, got, I'd, give,...",R&B
258118,"[Bad, Boys, Blue, ""I, Totally, Miss, You"", I, ...",Pop
258119,"[Forgive, me, for, the, things, That, I, never...",Pop
258120,"[The, day, they, found, a, cure, for, AIDS, Th...",Indie


In [7]:
data['labels'] = data['labels'].astype('category')
label_mapping = data['labels'].cat.categories
data['labels'] = data['labels'].cat.codes
X = data['texts']
y = data['labels']

In [8]:
type(X)

pandas.core.series.Series

In [9]:
def StratifiedKFold_feature_and_df_glove(df, feature_list, y_name):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  y = df[y_name]
  skf.get_n_splits(df[ feature_list ], y)

  splits = []

  for train_index, test_index in skf.split(df[ feature_list ], y):
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = df[ feature_list ].loc[train_index], df[ feature_list ].loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [10]:
def StratifiedKFold_feature_and_df(X, y):
  skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=1209)  # 20% for test set 
  #y = df[y_name]
  skf.get_n_splits(X, y)#df[ feature_list ]

  splits = []

  for train_index, test_index in skf.split(X, y):#df[ feature_list ]
      print("TRAIN:", train_index, "TEST:", test_index)
      X_train, X_test = X.loc[train_index], X.loc[test_index]
      y_train, y_test = y.loc[train_index], y.loc[test_index]
      splits.append({'X_train': X_train, 'X_test': X_test, 'y_train':y_train, 'y_test':y_test })
  return splits

In [11]:
#feature_list = ["texts"] #this is BOW and TF-IDF
#splits = StratifiedKFold_feature_and_df( data, feature_list, 'labels')
splits = StratifiedKFold_feature_and_df( X,y)

TRAIN: [     1      2      4 ... 258118 258120 258121] TEST: [     0      3      6 ... 258105 258112 258119]
TRAIN: [     0      1      2 ... 258119 258120 258121] TEST: [     7     15     20 ... 258110 258113 258114]
TRAIN: [     0      1      3 ... 258118 258119 258121] TEST: [     2      5     12 ... 258091 258116 258120]
TRAIN: [     0      2      3 ... 258119 258120 258121] TEST: [     1      8     10 ... 258102 258106 258118]
TRAIN: [     0      1      2 ... 258118 258119 258120] TEST: [     4     13     16 ... 258115 258117 258121]


In [12]:
len(splits)

5

In [13]:
split0=splits[0]
print(split0['X_train'].shape)
print(split0['y_train'].shape)
print(split0['X_test'].shape)
print(split0['y_test'].shape)

(206497,)
(206497,)
(51625,)
(51625,)


In [14]:
split0['X_train']

1         [Take, your, cold, hands, and, put, them, on, ...
2         [Are, you, ready, it's, time, for, war\nWe'll,...
4         [Do, you, believe, in, magic, in, a, young, gi...
5         [People, starin', at, me, as, they, wheel, me,...
7         [Tell, Mel, Shawn, to, come, in\nWord,, yo,, o...
                                ...                        
258116    [It's, complicated, It, always, is, That's, ju...
258117    [I'm, the, best, friend, he's, got, I'd, give,...
258118    [Bad, Boys, Blue, "I, Totally, Miss, You", I, ...
258120    [The, day, they, found, a, cure, for, AIDS, Th...
258121    [Fourth, of, July, has, come,, it's, custom, t...
Name: texts, Length: 206497, dtype: object

In [15]:
split0['y_train']

1         4
2         6
4         3
5         0
7         3
         ..
258116    1
258117    8
258118    7
258120    4
258121    7
Name: labels, Length: 206497, dtype: int8

In [16]:
counter_origin = Counter(df_genre_by_lang['Genre'])
counter_origin = sorted(counter_origin.items(), key= lambda k:k[1])
counter_origin

[('Electronic', 2664),
 ('Country', 2700),
 ('Hip-Hop', 3198),
 ('R&B', 3273),
 ('Indie', 7750),
 ('Folk', 8660),
 ('Jazz', 13974),
 ('Metal', 19943),
 ('Pop', 87405),
 ('Rock', 108555)]

In [17]:
counter_code = Counter(split0['y_train'])
counter_code= sorted(counter_code.items(), key= lambda k:k[1])
counter_code

[(1, 2131),
 (0, 2160),
 (3, 2558),
 (8, 2618),
 (4, 6200),
 (2, 6928),
 (5, 11180),
 (6, 15954),
 (7, 69924),
 (9, 86844)]

In [18]:
code2genre = {
    0: 'Country'
    , 1: 'Electronic'
    , 3:'Hip-Hop'
    , 8:'R&B'
    , 4: 'Indie'
    ,2: 'Folk'
    ,5: 'Jazz'
    ,6: 'Metal'
    ,7: 'Pop'
    ,9: 'Rock'
}
code2genre

{0: 'Country',
 1: 'Electronic',
 3: 'Hip-Hop',
 8: 'R&B',
 4: 'Indie',
 2: 'Folk',
 5: 'Jazz',
 6: 'Metal',
 7: 'Pop',
 9: 'Rock'}

### (Part1) Gensim Implementation

Parameter

sg=1 means to use skip-gram

min_count... is a threashhold that the algorithm requires a word need to appear at least x time ot be considered as part of skip-gram algorithm.

size... is the # of dimensions.

iterations ... ?


In [19]:
GLOVE_ZIP_FILE ="drive/MyDrive/music4all/glove.6B.zip"

!unzip drive/MyDrive/music4all/glove.6B.zip -d /content/

Archive:  drive/MyDrive/music4all/glove.6B.zip
  inflating: /content/glove.6B.50d.txt  
  inflating: /content/glove.6B.100d.txt  
  inflating: /content/glove.6B.200d.txt  
  inflating: /content/glove.6B.300d.txt  


### Gensim Word2Vec Class

Ref:https://datascience.stackexchange.com/questions/10695/how-to-initialize-a-new-word2vec-model-with-pre-trained-model-weights
Ref:https://gist.github.com/AbhishekAshokDubey/054af6f92d67d5ef8300fac58f59fcc9


In [20]:
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
from gensim.models import Word2Vec, Phrases, phrases, KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec

class GensimWord2VecVectorizer(BaseEstimator, TransformerMixin):
    """
    Word vectors are averaged across to create the document-level vectors/features.
    gensim's own gensim.sklearn_api.W2VTransformer doesn't support out of vocabulary words,
    hence we roll out our own.
    All the parameters are gensim.models.Word2Vec's parameters.
    https://radimrehurek.com/gensim/models/word2vec.html#gensim.models.word2vec.Word2Vec
    """
    
    def __init__(self, size=100, alpha=0.025, window=5, min_count=5, max_vocab_size=None,
                 sample=0.001, seed=1, workers=3, min_alpha=0.0001, sg=0, hs=0, negative=5,
                 ns_exponent=0.75, cbow_mean=1, hashfxn=hash, iter=5, null_word=0,
                 trim_rule=None, sorted_vocab=1, batch_words=10000, compute_loss=False,
                 callbacks=(), max_final_vocab=None):
        self.size = size
        self.alpha = alpha
        self.window = window
        self.min_count = min_count
        self.max_vocab_size = max_vocab_size
        self.sample = sample
        self.seed = seed
        self.workers = workers
        self.min_alpha = min_alpha
        self.sg = sg
        self.hs = hs
        self.negative = negative
        self.ns_exponent = ns_exponent
        self.cbow_mean = cbow_mean
        self.hashfxn = hashfxn
        self.iter = iter
        self.null_word = null_word
        self.trim_rule = trim_rule
        self.sorted_vocab = sorted_vocab
        self.batch_words = batch_words
        self.compute_loss = compute_loss
        self.callbacks = callbacks
        self.max_final_vocab = max_final_vocab

    def fit(self, X, y=None):
        self.model_ = Word2Vec(
            sentences=X, corpus_file=None,
            size=self.size, alpha=self.alpha, window=self.window, min_count=self.min_count,
            max_vocab_size=self.max_vocab_size, sample=self.sample, seed=self.seed,
            workers=self.workers, min_alpha=self.min_alpha, sg=self.sg, hs=self.hs,
            negative=self.negative, ns_exponent=self.ns_exponent, cbow_mean=self.cbow_mean,
            hashfxn=self.hashfxn, iter=self.iter, null_word=self.null_word,
            trim_rule=self.trim_rule, sorted_vocab=self.sorted_vocab, batch_words=self.batch_words,
            compute_loss=self.compute_loss, callbacks=self.callbacks,
            max_final_vocab=self.max_final_vocab)

        #My code added
        #google_wv = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)
        #Ref: https://radimrehurek.com/gensim/scripts/glove2word2vec.html
        

        print("Created glove_wv.")
        print("Before build_vocab(X), check if Word2Vec(sentences=X) builds a vocab : ", len(self.model_.wv.vocab))

        #self.model_.build_vocab(X)
        training_examples_count = self.model_.corpus_count
        print("original training_examples_count:", training_examples_count)
        print("Before merge, vocab: ", len(self.model_.wv.vocab))

        if self.size ==100:
          # load the Stanford GloVe model
          glove_input_file = 'glove.6B.100d.txt'
          word2vec_output_file = 'glove.6B.100d.txt.word2vec'
          glove2word2vec(glove_input_file, word2vec_output_file)
          filename = word2vec_output_file
          glove_wv = KeyedVectors.load_word2vec_format(filename, binary=False)
        elif self.size ==200:
          glove_input_file = 'glove.6B.200d.txt'
          word2vec_output_file = 'glove.6B.200d.txt.word2vec'
          glove2word2vec(glove_input_file, word2vec_output_file)
          filename = word2vec_output_file
          glove_wv = KeyedVectors.load_word2vec_format(filename, binary=False)
        elif self.size ==300:
          glove_input_file = 'glove.6B.300d.txt'
          word2vec_output_file = 'glove.6B.300d.txt.word2vec'
          glove2word2vec(glove_input_file, word2vec_output_file)
          filename = word2vec_output_file
          glove_wv = KeyedVectors.load_word2vec_format(filename, binary=False)


        self.model_.build_vocab([list(glove_wv.vocab.keys())], update=True)
        print("Updated build_vocab by Glove.")
        #self.model_.intersect_word2vec_format(glove_file, binary=False, lockf=1.0)#try this
        self.model_.intersect_word2vec_format(filename, binary=False, lockf=1.0)
        print("intersect completed.")
        self.model_.train(X, total_examples=training_examples_count, epochs=self.model_.iter)

        print("After merge, vocab: ", len(self.model_.wv.vocab))
        return self

    def transform(self, X):
        X_embeddings = np.array([self._get_embedding(words) for words in X])
        return X_embeddings

    def _get_embedding(self, words):
        valid_words = [word for word in words if word in self.model_.wv.vocab]
        if valid_words:
            embedding = np.zeros((len(valid_words), self.size), dtype=np.float32)
            for idx, word in enumerate(valid_words):
                embedding[idx] = self.model_.wv[word]

            return np.mean(embedding, axis=0)
        else:
            return np.zeros(self.size)

In [21]:
def X_train_test_prepare(X_train, X_test):
  first_index =X_train.index[0]
  if type(X_train[first_index]) is not list:
    for idx, lyrics in X_train.items():
      X_train[idx] = lyrics.split(' ')

  first_index =X_test.index[0]
  if type(X_test[first_index]) is not list:
    for idx, lyrics in X_test.items():
      X_test[idx] = lyrics.split(' ')

  return X_train, X_test

### Model: Skipgram with Negative sampling. Run KFold and take mean of the balanced accuracy. use the best parameter from the music4all experiment, using XGB


In [22]:
balanced_accuracy = []
DIM = 200

In [ ]:
split0['X_train']

1         [Take, your, cold, hands, and, put, them, on, ...
2         [Are, you, ready, it's, time, for, war\nWe'll,...
4         [Do, you, believe, in, magic, in, a, young, gi...
5         [People, starin', at, me, as, they, wheel, me,...
7         [Tell, Mel, Shawn, to, come, in\nWord,, yo,, o...
                                ...                        
258116    [It's, complicated, It, always, is, That's, ju...
258117    [I'm, the, best, friend, he's, got, I'd, give,...
258118    [Bad, Boys, Blue, "I, Totally, Miss, You", I, ...
258120    [The, day, they, found, a, cure, for, AIDS, Th...
258121    [Fourth, of, July, has, come,, it's, custom, t...
Name: texts, Length: 206497, dtype: object

### First Fold

In [23]:
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix

#0. get X_train, X_test
split0=splits[0]
split0['X_train'], split0['X_test'] = X_train_test_prepare(split0['X_train'], split0['X_test'])
print("=========split: " , 0, "=================")
#1.create model and clf
gensim_word2vec_tr_temp = GensimWord2VecVectorizer(size=DIM, min_count=5, sg=1, alpha=0.025, iter=1, sample=1e-05)

#Note: from parameter tuning we found n_estimator=200 is best so far.
xgb_clf_SMOTE = XGBClassifier(learning_rate=0.01, n_estimators=200, n_jobs=-1)

#2.create word embedding and vector representation for each song
gensim_word2vec_tr_temp = gensim_word2vec_tr_temp.fit(split0['X_train'])

=========split:  0 =================
Created glove_wv.
Before build_vocab(X), check if Word2Vec(sentences=X) builds a vocab :  297432
original training_examples_count: 206497
Before merge, vocab:  297432
Updated build_vocab by Glove.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


intersect completed.
After merge, vocab:  297432


Continue from the last cell

In [24]:
#2-1.I want to add vocab from Gloeve
X_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_train'])
print(len(X_embeddings_temp))
#3.SMOTE on train
X_resampled_temp, y_resampled_temp = SMOTE().fit_resample(X_embeddings_temp, split0['y_train'])
print(Counter(y_resampled_temp))

206497
Counter({4: 86844, 6: 86844, 3: 86844, 0: 86844, 7: 86844, 5: 86844, 9: 86844, 1: 86844, 2: 86844, 8: 86844})


In [25]:
#4.train
xgb_clf_SMOTE.fit(X_resampled_temp, y_resampled_temp)

import pickle
filename = DIR + 'Glove_XGB_kaggle_split0.sav'
pickle.dump(xgb_clf_SMOTE, open(filename, 'wb'))

In [27]:
X_test_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_test'])
y_test_pred_SMOTE_temp = xgb_clf_SMOTE.predict(X_test_embeddings_temp)
print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) )
print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))

              precision    recall  f1-score   support

           0       0.07      0.48      0.13       540
           1       0.06      0.24      0.10       533
           2       0.09      0.28      0.14      1732
           3       0.12      0.70      0.20       640
           4       0.05      0.14      0.07      1550
           5       0.22      0.52      0.31      2794
           6       0.29      0.55      0.38      3989
           7       0.53      0.26      0.35     17481
           8       0.03      0.22      0.05       655
           9       0.57      0.12      0.20     21711

    accuracy                           0.24     51625
   macro avg       0.20      0.35      0.19     51625
weighted avg       0.46      0.24      0.26     51625

0.3506699630220461


In [ ]:
#4.train
xgb_clf_SMOTE.fit(X_resampled_temp, y_resampled_temp)

#5.Create test embedding and then predict
#Note: this is because test embedding shape changes for each loop
X_test_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_test'])
y_test_pred_SMOTE_temp = clf_svm_temp.predict(X_test_embeddings_temp)
print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) )
print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))
balanced_accuracy.append(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))

with open(DIR+ "Glove_SVM_base_kaggle_log.txt", "a") as f:
  print("=========split: " , i,  "==n_estimator: 200", "=learn rate:0.01", "=================", file=f)
  print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)
  print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)

if (i, DIM, kernel_choice,gamma_choice, C_choice ) not in McNemar:
  McNemar[(i, DIM, kernel_choice,gamma_choice, C_choice )] = []
for ground_truth, pred in zip(split0['y_test'], y_test_pred_SMOTE_temp):
  if ground_truth == pred:
    McNemar[(i, DIM, kernel_choice,gamma_choice, C_choice )].append(True)
  else:
    McNemar[(i, DIM, kernel_choice,gamma_choice, C_choice )].append(False)

NameError: ignored

Took 1H46 to fit

Resume from the last cell, predict

In [ ]:
McNemar={}

In [ ]:
y_test_pred_SMOTE_temp = xgb_clf_SMOTE.predict(X_test_embeddings_temp)
print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) )
print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))
balanced_accuracy.append(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))

with open(DIR+ "Glove_XGB_kaggle_log.txt", "a") as f:
  print("=========split: " , 0,  "==n_estimator: 200", "=learn rate:0.01", "=================", file=f)
  print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)
  print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)

              precision    recall  f1-score   support

           0       0.06      0.44      0.10       540
           1       0.05      0.24      0.09       533
           2       0.09      0.27      0.14      1732
           3       0.12      0.70      0.20       640
           4       0.05      0.16      0.08      1550
           5       0.24      0.51      0.32      2794
           6       0.29      0.54      0.37      3989
           7       0.55      0.25      0.35     17481
           8       0.03      0.23      0.05       655
           9       0.57      0.11      0.18     21711

    accuracy                           0.23     51625
   macro avg       0.20      0.35      0.19     51625
weighted avg       0.47      0.23      0.25     51625

0.34556170501658384


### Second Fold

In [ ]:
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix

#0. get X_train, X_test
split0=splits[1]
split0['X_train'], split0['X_test'] = X_train_test_prepare(split0['X_train'], split0['X_test'])
print("=========split: " , 1, "=================")
#1.create model and clf
gensim_word2vec_tr_temp = GensimWord2VecVectorizer(size=DIM, min_count=5, sg=1, alpha=0.025, iter=1, sample=1e-05)

#Note: from parameter tuning we found n_estimator=200 is best so far.
xgb_clf_SMOTE = XGBClassifier(learning_rate=0.01, n_estimators=200, n_jobs=-1)

#2.create word embedding and vector representation for each song
gensim_word2vec_tr_temp = gensim_word2vec_tr_temp.fit(split0['X_train'])

=========split:  1 =================
Created glove_wv.
Before build_vocab(X), check if Word2Vec(sentences=X) builds a vocab :  297235
original training_examples_count: 206497
Before merge, vocab:  297235
Updated build_vocab by Glove.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


intersect completed.
After merge, vocab:  297235


In [ ]:
#2-1.I want to add vocab from Gloeve
X_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_train'])
print(len(X_embeddings_temp))
#3.SMOTE on train
X_resampled_temp, y_resampled_temp = SMOTE().fit_resample(X_embeddings_temp, split0['y_train'])
print(Counter(y_resampled_temp))

206497
Counter({3: 86844, 4: 86844, 6: 86844, 7: 86844, 0: 86844, 9: 86844, 8: 86844, 1: 86844, 2: 86844, 5: 86844})


In [ ]:
#4.train
xgb_clf_SMOTE.fit(X_resampled_temp, y_resampled_temp)

#5.Create test embedding and then predict
#Note: this is because test embedding shape changes for each loop
X_test_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_test'])
y_test_pred_SMOTE_temp = xgb_clf_SMOTE.predict(X_test_embeddings_temp)
print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) )
print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))
balanced_accuracy.append(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))

with open(DIR+ "Glove_SVM_base_kaggle_log.txt", "a") as f:
  print("=========split: " , 1,  "==n_estimator: 200", "=learn rate:0.01", "=================", file=f)
  print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)
  print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)

              precision    recall  f1-score   support

           0       0.07      0.47      0.12       540
           1       0.06      0.27      0.10       533
           2       0.08      0.28      0.12      1732
           3       0.11      0.70      0.19       640
           4       0.05      0.13      0.07      1550
           5       0.22      0.52      0.31      2795
           6       0.29      0.52      0.38      3988
           7       0.54      0.27      0.36     17481
           8       0.03      0.22      0.05       655
           9       0.57      0.12      0.19     21711

    accuracy                           0.24     51625
   macro avg       0.20      0.35      0.19     51625
weighted avg       0.46      0.24      0.26     51625

0.34968158062765864


Took 1H46 to fit

### Third Fold

In [ ]:
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix

#0. get X_train, X_test
split0=splits[2]
split0['X_train'], split0['X_test'] = X_train_test_prepare(split0['X_train'], split0['X_test'])
print("=========split: " , 2, "=================")
#1.create model and clf
gensim_word2vec_tr_temp = GensimWord2VecVectorizer(size=DIM, min_count=5, sg=1, alpha=0.025, iter=1, sample=1e-05)

#Note: from parameter tuning we found n_estimator=200 is best so far.
xgb_clf_SMOTE = XGBClassifier(learning_rate=0.01, n_estimators=200, n_jobs=-1)

#2.create word embedding and vector representation for each song
gensim_word2vec_tr_temp = gensim_word2vec_tr_temp.fit(split0['X_train'])

=========split:  2 =================
Created glove_wv.
Before build_vocab(X), check if Word2Vec(sentences=X) builds a vocab :  297288
original training_examples_count: 206498
Before merge, vocab:  297288
Updated build_vocab by Glove.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


intersect completed.
After merge, vocab:  297288


In [ ]:
#2-1.I want to add vocab from Gloeve
X_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_train'])
print(len(X_embeddings_temp))
#3.SMOTE on train
X_resampled_temp, y_resampled_temp = SMOTE().fit_resample(X_embeddings_temp, split0['y_train'])
print(Counter(y_resampled_temp))

206498
Counter({3: 86844, 4: 86844, 7: 86844, 0: 86844, 6: 86844, 5: 86844, 9: 86844, 8: 86844, 1: 86844, 2: 86844})


In [ ]:
#4.train
xgb_clf_SMOTE.fit(X_resampled_temp, y_resampled_temp)

#5.Create test embedding and then predict
#Note: this is because test embedding shape changes for each loop
X_test_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_test'])
y_test_pred_SMOTE_temp = xgb_clf_SMOTE.predict(X_test_embeddings_temp)
print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) )
print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))
balanced_accuracy.append(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))

with open(DIR+ "Glove_XGB_kaggle_log.txt", "a") as f:
  print("=========split: " , 2,  "==n_estimator: 200", "=learn rate:0.01", "=================", file=f)
  print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)
  print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)

              precision    recall  f1-score   support

           0       0.07      0.41      0.12       540
           1       0.07      0.27      0.11       532
           2       0.09      0.29      0.14      1732
           3       0.11      0.74      0.19       640
           4       0.05      0.17      0.08      1550
           5       0.22      0.50      0.30      2795
           6       0.29      0.53      0.37      3988
           7       0.57      0.25      0.34     17481
           8       0.04      0.24      0.06       655
           9       0.57      0.15      0.24     21711

    accuracy                           0.25     51624
   macro avg       0.21      0.35      0.20     51624
weighted avg       0.47      0.25      0.28     51624

0.35412064341860783


Took 1H46 to fit

In [ ]:
import pickle
filename = DIR + 'checkpoints/Glove_XGB_kaggle_split2.sav'
pickle.dump(xgb_clf_SMOTE, open(filename, 'wb'))

### Fourth Fold

In [ ]:
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix

#0. get X_train, X_test
split0=splits[3]
split0['X_train'], split0['X_test'] = X_train_test_prepare(split0['X_train'], split0['X_test'])
print("=========split: " , 3, "=================")
#1.create model and clf
gensim_word2vec_tr_temp = GensimWord2VecVectorizer(size=DIM, min_count=5, sg=1, alpha=0.025, iter=1, sample=1e-05)

#Note: from parameter tuning we found n_estimator=200 is best so far.
xgb_clf_SMOTE = XGBClassifier(learning_rate=0.01, n_estimators=200, n_jobs=-1)

#2.create word embedding and vector representation for each song
gensim_word2vec_tr_temp = gensim_word2vec_tr_temp.fit(split0['X_train'])

=========split:  3 =================
Created glove_wv.
Before build_vocab(X), check if Word2Vec(sentences=X) builds a vocab :  296978
original training_examples_count: 206498
Before merge, vocab:  296978
Updated build_vocab by Glove.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


intersect completed.
After merge, vocab:  296978


In [ ]:
#2-1.I want to add vocab from Gloeve
X_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_train'])
print(len(X_embeddings_temp))
#3.SMOTE on train
X_resampled_temp, y_resampled_temp = SMOTE().fit_resample(X_embeddings_temp, split0['y_train'])
print(Counter(y_resampled_temp))

206498
Counter({3: 86844, 6: 86844, 7: 86844, 0: 86844, 5: 86844, 9: 86844, 4: 86844, 8: 86844, 1: 86844, 2: 86844})


In [ ]:
#4.train
xgb_clf_SMOTE.fit(X_resampled_temp, y_resampled_temp)

#5.Create test embedding and then predict
#Note: this is because test embedding shape changes for each loop
X_test_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_test'])
y_test_pred_SMOTE_temp = xgb_clf_SMOTE.predict(X_test_embeddings_temp)
print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) )
print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))
balanced_accuracy.append(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))

with open(DIR+ "Glove_XGB_kaggle_log.txt", "a") as f:
  print("=========split: " , 3,  "==n_estimator: 200", "=learn rate:0.01", "=================", file=f)
  print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)
  print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)

              precision    recall  f1-score   support

           0       0.08      0.49      0.13       540
           1       0.07      0.28      0.11       533
           2       0.09      0.28      0.13      1732
           3       0.13      0.71      0.22       639
           4       0.06      0.16      0.09      1550
           5       0.23      0.52      0.31      2795
           6       0.25      0.58      0.35      3989
           7       0.55      0.23      0.33     17481
           8       0.04      0.28      0.06       654
           9       0.58      0.11      0.18     21711

    accuracy                           0.23     51624
   macro avg       0.21      0.37      0.19     51624
weighted avg       0.47      0.23      0.25     51624

0.3655926343925764


In [ ]:
import pickle
filename = DIR + 'checkpoints/Glove_XGB_kaggle_split3.sav'
pickle.dump(xgb_clf_SMOTE, open(filename, 'wb'))

### Fifth Fold

In [ ]:
from sklearn import svm
from xgboost import XGBClassifier
from sklearn.metrics import balanced_accuracy_score
from imblearn.over_sampling import SMOTE
from sklearn.metrics import classification_report, confusion_matrix

#0. get X_train, X_test
split0=splits[4]
split0['X_train'], split0['X_test'] = X_train_test_prepare(split0['X_train'], split0['X_test'])
print("=========split: " , 4, "=================")
#1.create model and clf
gensim_word2vec_tr_temp = GensimWord2VecVectorizer(size=DIM, min_count=5, sg=1, alpha=0.025, iter=1, sample=1e-05)

#Note: from parameter tuning we found n_estimator=200 is best so far.
xgb_clf_SMOTE = XGBClassifier(learning_rate=0.01, n_estimators=200, n_jobs=-1)

#2.create word embedding and vector representation for each song
gensim_word2vec_tr_temp = gensim_word2vec_tr_temp.fit(split0['X_train'])

=========split:  4 =================
Created glove_wv.
Before build_vocab(X), check if Word2Vec(sentences=X) builds a vocab :  297552
original training_examples_count: 206498
Before merge, vocab:  297552
Updated build_vocab by Glove.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:96: DeprecationWarning: Call to deprecated `iter` (Attribute will be removed in 4.0.0, use self.epochs instead).


intersect completed.
After merge, vocab:  297552


In [ ]:
#2-1.I want to add vocab from Gloeve
X_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_train'])
print(len(X_embeddings_temp))
#3.SMOTE on train
X_resampled_temp, y_resampled_temp = SMOTE().fit_resample(X_embeddings_temp, split0['y_train'])
print(Counter(y_resampled_temp))

206498
Counter({3: 86844, 4: 86844, 6: 86844, 7: 86844, 0: 86844, 5: 86844, 9: 86844, 8: 86844, 1: 86844, 2: 86844})


In [ ]:
#4.train
xgb_clf_SMOTE.fit(X_resampled_temp, y_resampled_temp)

#5.Create test embedding and then predict
#Note: this is because test embedding shape changes for each loop
X_test_embeddings_temp = gensim_word2vec_tr_temp.transform(split0['X_test'])
y_test_pred_SMOTE_temp = xgb_clf_SMOTE.predict(X_test_embeddings_temp)
print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) )
print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))
balanced_accuracy.append(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp))

with open(DIR+ "Glove_XGB_base_kaggle_log.txt", "a") as f:
  print("=========split: " , 4,  "==n_estimator: 200", "=learn rate:0.01", "=================", file=f)
  print(classification_report(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)
  print(balanced_accuracy_score(split0['y_test'], y_test_pred_SMOTE_temp) , file=f)

              precision    recall  f1-score   support

           0       0.08      0.48      0.13       540
           1       0.07      0.29      0.11       533
           2       0.08      0.26      0.13      1732
           3       0.12      0.68      0.20       639
           4       0.05      0.12      0.07      1550
           5       0.22      0.51      0.31      2795
           6       0.29      0.57      0.38      3989
           7       0.55      0.27      0.36     17481
           8       0.03      0.21      0.05       654
           9       0.58      0.13      0.22     21711

    accuracy                           0.25     51624
   macro avg       0.21      0.35      0.20     51624
weighted avg       0.47      0.25      0.27     51624

0.35396073222743846


Save model

In [ ]:
import pickle
filename = DIR + 'checkpoints/Glove_XGB_kaggle_split4.sav'
pickle.dump(xgb_clf_SMOTE, open(filename, 'wb'))